In [1]:
import numpy as np
import healpy as hp
from astropy.io import fits
from scipy.optimize import least_squares
from tqdm.notebook import tqdm
import dynesty
from dynesty import utils as dyfunc
from multiprocessing import Pool




#from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcdefaults()
from matplotlib import font_manager
from matplotlib import rcParams
from matplotlib import rc
from matplotlib import colors

'''rcParams['mathtext.rm'] = 'Computer Modern'
rcParams['text.usetex'] = True
rcParams['font.family'] = 'serif'

font_manager.findfont('serif', rebuild_if_missing=True)
fontsize = 14
rcParams.update({'font.size' : fontsize})'''

#My own files 
from MyUnits import *
from fake_dict import fake_dict

HomeDir = './'
DataDir = HomeDir + 'template_data/'

# Process fake data

In [2]:
## frequencies of maps
nu_list = np.asarray(list(fake_dict.keys()))
num_maps = len(nu_list)
## map properties
rms_list = np.asarray([fake_dict.get(f).get('rms') for f in nu_list])
calib_list = np.asarray([fake_dict.get(f).get('calib') for f in nu_list])
zp_list = np.asarray([fake_dict.get(f).get('zero_pt') for f in nu_list])
resol_list = np.asarray([fake_dict.get(f).get('resol') for f in nu_list])
## template coefficients for fake data
c_synch_list = np.asarray([fake_dict.get(f).get('c_synch') for f in nu_list])
c_ff_list = np.asarray([fake_dict.get(f).get('c_ff') for f in nu_list])
c_src_list = np.asarray([fake_dict.get(f).get('c_src') for f in nu_list])
c_struct_list = np.asarray([fake_dict.get(f).get('c_struct') for f in nu_list])


# initial resolution of fake_data
i_res = 64 

# resolution of fit
f_res = 16

npix = hp.nside2npix(i_res)
f_res_pix = hp.nside2npix(f_res)

# nmap x npix array of skymaps
skymaps = np.zeros((len(nu_list), npix))
pt_maps = np.zeros((len(nu_list), npix)) # treat point source maps separately
nanloc = np.zeros((len(nu_list), npix), dtype='bool') # keep track of nan locations
inv_cov = np.zeros((len(nu_list), f_res_pix))

print(rms_list)

[3.00e+03 9.63e+02 3.00e+02 5.26e+02 3.65e+02 2.08e+02 1.12e+02 1.20e+00
 1.70e-02 9.01e-03 7.65e-03]


# Import model

In [3]:
# Get rid of slice |b| < b_min degrees
def remove_sky(skymap, b_min, l_min = 0, is_nest = False):
    nside = hp.npix2nside(len(skymap))
    skymap_cp = np.copy(skymap)
    lon, lat = hp.pix2ang(nside, np.arange(len(skymap_cp)), nest = is_nest, lonlat = True)
    skymap_cp[np.abs(lat) < b_min] = np.nan
    skymap_cp[lon < l_min] = np.nan
    skymap_cp[lon > 360 - l_min] = np.nan
    #print(len(skymap_cp[~np.isnan(skymap_cp)]))
    return skymap_cp

# Brightness temperature of CMB
def cmb_bt(freq):
    return (2.72548 * Kelvin) *(2 * np.pi * freq/(2.72548 * Kelvin)) /(np.exp(2 * np.pi * freq/(2.72548 * Kelvin))-1)

## Masking 

# minimum latitude and longitudes we consider
B_MIN = 10
L_MIN = 0

# initial resolution of fake_data
i_res = 64 

# resolution of fit
f_res = 16

npix = hp.nside2npix(i_res)
f_res_pix = hp.nside2npix(f_res)

pix_list = np.arange(f_res_pix, dtype = 'float')
pix_list = remove_sky(pix_list, B_MIN, L_MIN)
pix_list = pix_list[~np.isnan(pix_list)]
pix_list = np.asarray(pix_list, dtype = 'int')

num_pix_r = len(pix_list[~np.isnan(pix_list)]); pix_list


#number of b bins per hemisphere
num_b = 2
b_bins = np.arange(-90, 90, 180/(num_b))
# latitudes
_, lat = hp.pix2ang(f_res, pix_list, nest = False, lonlat = True)
# b bin assignments
b_assgn = np.digitize(lat, bins = b_bins)
#b_bins = np.concatenate((b_bins, [90]))
b_bins, b_assgn

csc_i = 1/np.sin(np.abs(lat) * degree)

# load source template 

pt_src = np.load(DataDir + 'pt_src.npy')
vlbi_src = np.load(DataDir + 'vlbi_src_mask.npy')
pt_src_dg = hp.ud_grade(pt_src, f_res)
vlbi_src_dg = hp.ud_grade(vlbi_src, f_res)

nu_ = np.asarray([float(n) for n in nu_list]) / 310.0 # reference frequency is 310 MHz
nu_arr = np.broadcast_to(nu_, (len(csc_i),len(nu_list))).T

# dimensionality of our problem : 2 + 2 * num_b

ndim = 2 + 2 * num_b + 1
print ('ndim = ' + str(ndim))


def loglike(x):
    # extragalactic parameters
    T_CMB =  x[0] #2.722
    T_E =  x[1] #30.4 #
    beta_E = x[2] #-2.58 # 
    
    # galactic parameters
    T_G  =  x[3 : (2 + num_b)+1] #np.asarray([9.65,8.06]) #
    
    beta_G = x[(2 + num_b)+1 : (2 + 2 * num_b)+1] # np.asarray([-2.58, -2.56])#
    
    # isotropic component
    TE_i = T_E * np.ones(num_pix_r) 
    nu_beta_E = np.power(nu_, beta_E)
    TE_i_q = np.outer(nu_beta_E, TE_i)

    # galactic component
    TG_csc_i = T_G[b_assgn-1] * csc_i
    nu_beta_G = np.power(nu_arr, beta_G[b_assgn-1])
    TG_i_q = TG_csc_i * nu_beta_G
    
    # residual array
    res_arr = inv_cov_r * (skymaps_r - (T_CMB + TE_i_q + TG_i_q))**2
    
    # contract over pixel and map indices
    return -0.5 * np.einsum('qi->', res_arr)



'''def ptform(u):
    return scale * u + shift'''

ndim = 7


'def ptform(u):\n    return scale * u + shift'

# Read in the files 

In [4]:
### Read in maps

for i_f, f in enumerate(nu_list):
    # load in real_map, 
    real_map = np.load(fake_dict.get(f).get('skymap'))
    real_map = hp.ud_grade(real_map, i_res, pess=True)
    real_map[real_map < 0] = np.nan
    nanloc[i_f] = (np.isnan(real_map))

    # load fake map components (except)
    synch_map = hp.ud_grade(np.load(fake_dict.get(f).get('synch_map')), i_res, pess = True)
    ff_map = hp.ud_grade(np.load(fake_dict.get(f).get('ff_map')), i_res, pess = True)
    src_map = hp.ud_grade(np.load(fake_dict.get(f).get('src_map')), i_res, pess = True)
    struct_map = hp.ud_grade(np.load(fake_dict.get(f).get('struct_map')), i_res, pess = True)

    # combine templates, treat point sources separately 
    data = c_synch_list[i_f] * synch_map + c_ff_list[i_f] * ff_map  + c_struct_list[i_f] * struct_map
    pt = c_src_list[i_f] * src_map

    skymaps[i_f] = data 
    pt_maps[i_f] = pt

## Inject signal

In [5]:
# Define injection parameters 

### Extragalactic temperature(s) at 310 MHz (in K)
T_eg_list = np.asarray([50])#np.asarray([10,20, 25,30, 35, 40, 45,50, 55,60, 65,70,80,90])
### Spectral index for T_eg
beta_eg = -2.58
### number of iterations per temperature
n_iter = 1

means_teg = np.zeros((len(T_eg_list), n_iter))
means_beg = np.zeros((len(T_eg_list), n_iter))
means_cmb = np.zeros((len(T_eg_list), n_iter))

errs_teg = np.zeros((len(T_eg_list), n_iter))
errs_beg = np.zeros((len(T_eg_list), n_iter))
errs_cmb = np.zeros((len(T_eg_list), n_iter))

In [6]:
scale = np.zeros(ndim) # size of prior
shift = np.zeros(ndim) # location of prior

scale[0] = 0.0025
scale[1] = 70
scale[2] = 0.3
scale[3: (2 + num_b)+1] = 0.0025
scale[(2 + num_b)+1 : (2 + 2 * num_b)+1] = 0.0025

shift[0] = -0.0125
shift[1] = 0
shift[2] = -2.7
shift[3: (2 + num_b)+1] = -0.00125
shift[(2 + num_b)+1 : (2 + 2 * num_b)+1] = -0.00125

def ptform(u):
        return scale * u + shift

In [7]:
T_eg = T_eg_list[0]
j = 0

### Inject temperatures 
inj_maps_h = np.outer(nu_**beta_eg, T_eg * np.ones(npix)) # high-res
inj_maps_h = inj_maps_h #+ skymaps # inject signal 

cmb_list = 2.725 * np.ones(npix) #np.outer(np.asarray([cmb_bt(float(f) * MHz) / Kelvin for f in nu_list]), np.ones(npix))
inj_maps_h = inj_maps_h #+ cmb_list # inject CMB

inj_maps = np.zeros((len(nu_list), f_res_pix)) #downgraded maps

## convolve with error and make inv cov "matrix"
for i_f, f in enumerate(nu_list):
    ## load map of variances
    vars = rms_list[i_f]**2 + (calib_list[i_f] * inj_maps_h[i_f])**2 + zp_list[i_f]**2 # np.load(fake_dict.get(f).get('err_map')) 
    ### sample from gaussian 
    inj_maps_h[i_f] = np.random.normal(inj_maps_h[i_f], np.sqrt(vars))
    pt_maps[i_f][pt_maps[i_f] > 0] = np.random.normal(pt_maps[i_f][pt_maps[i_f] > 0], 
                                            np.sqrt((calib_list[i_f] * pt_maps[i_f][pt_maps[i_f] > 0])**2))

    ### smooth with gaussian beam (make fluctuations correlated)
    smoothed = inj_maps_h[i_f] #hp.sphtfunc.smoothing(inj_maps_h[i_f], fwhm = 4 * degree)
    meas_map = smoothed + pt_maps[i_f]
    meas_map[nanloc[i_f]] = np.nan # set nans to reflect true map
    inj_maps_h[i_f] = meas_map
    
    inj_maps[i_f] = hp.ud_grade(meas_map, f_res, pess = True)  
    inj_maps[i_f][np.isnan(inj_maps[i_f])] = 0
    inj_maps[i_f][inj_maps[i_f] < 0] = 0  

    vars[vars <= 0] = np.nan
    vars = hp.ud_grade(vars, f_res, pess = True)
    inv_cov[i_f] = 1/vars
    # Important: set incomplete data to 0, NOT np.nan, so we can naturally exclude this from the fit
    inv_cov[i_f][np.isnan(inv_cov[i_f])] = 0
# chop up the maps so only 4 quadrants remain
# mask out point sources according to pt_tmpl

for i in range(len(nu_list)):
    inj_maps[i] = remove_sky(inj_maps[i], B_MIN, L_MIN)
    #skymaps[i][pt_src_dg > 0] = np.nan
    #skymaps[i][vlbi_src_dg > 0] = np.nan

    inv_cov[i] = remove_sky(inv_cov[i], B_MIN, L_MIN)
    #inv_cov[i][pt_src_dg > 0] = np.nan
    #inv_cov[i][vlbi_src_dg > 0] = np.nan


#exclude nan vals

skymaps_r = np.zeros((len(nu_list), num_pix_r))
inv_cov_r = np.zeros((len(nu_list), num_pix_r))


for i_f, f in enumerate(nu_list):
    skymaps_r[i_f] = inj_maps[i_f][~np.isnan(inj_maps[i_f])]
    inv_cov_r[i_f] = inv_cov[i_f][~np.isnan(inv_cov[i_f])]

num_cores = 40
pool = Pool(num_cores)
# "Dynamic" nested sampling.
dsampler = dynesty.DynamicNestedSampler(loglike, ptform, ndim, pool = pool, queue_size = num_cores, sample='auto')
dsampler.run_nested(print_progress=True)
results = dsampler.results

# Extract sampling results.
samples = results.samples  # samples
weights = np.exp(results.logwt - results.logz[-1])  # normalized weights

# Compute 1 sigma error bars
quantiles = [dyfunc.quantile(samps, [0.159,0.5,0.841], weights=weights)
            for samps in samples.T]

# Compute weighted mean and covariance.
mean, cov = dyfunc.mean_and_cov(samples, weights)
print(mean)

means_cmb[i_t][j] = mean[0]
errs_cmb[i_t][j] = max(quantiles[0][2] - quantiles[0][1], quantiles[0][1] - quantiles[0][0])

means_teg[i_t][j] = mean[1]
errs_teg[i_t][j] = max(quantiles[1][2] - quantiles[1][1], quantiles[1][1] - quantiles[1][0])

means_beg[i_t][j] = mean[2]
errs_beg[i_t][j] = max(quantiles[2][2] - quantiles[2][1], quantiles[2][1] - quantiles[2][0])
print(T_eg)

20962it [02:18, 65.42it/s, batch: 0 | bound: 277 | nc: 100 | ncall: 228473 | eff(%):  9.155 | loglstar:   -inf < -217724.711 <    inf | logz: -217766.423 +/-  0.283 | dlogz:  6.216 >  0.010] 

In [ ]:
means_teg, errs_teg

(array([[45.00028184]]), array([[0.00032383]]))

In [ ]:
means_beg, errs_beg

(array([[-2.47999782]]), array([[2.45275647e-06]]))

In [ ]:
means_cmb

array([[7.08991328e-07]])

In [ ]:
np.save('mean_teg.npy', means_teg)
np.save('mean_beg.npy', means_beg)
np.save('mean_cmb.npy', means_cmb)

In [ ]:
np.save('errs_teg.npy', errs_teg)
np.save('errs_beg.npy', errs_beg)
np.save('errs_cmb.npy', errs_cmb)